# Neural Networks from Scratch

Create a simple neural network from scratch using Python.

## Imports

In [33]:
from abc import abstractmethod
import numpy as np

## Auxiliar functions

In [34]:

def sigmoid(x):
  return 1/(1+np.exp(-x))

def deriv_sigmoid(x):
  return sigmoid(x)-(1-sigmoid(x))

deriv_activation_functions={
    sigmoid: deriv_sigmoid
}

def connect_layer_forward(input_nodes, neurons):
  for input_node in input_nodes:
    for neuron in neurons:
      input_node.connect_forward(neuron)

## Classes

In [35]:
class Node():
  """
  Iterface that represent a single node element of a neural network
  """
  @abstractmethod
  def get_output(self):
    """
    Returns the output value of a node
    """
    pass

  @abstractmethod
  def backpropagate(self):
    """
    Performs backpropagation process recursively
    """
    pass

  @abstractmethod
  def connect_forward(self,output_node):
    """
    Connects the current node to a forward node.
    """
    pass

In [36]:
class Neuron(Node):
  """
  Represents a single neuron in a neural network 
  """
  def __init__(self,  activation_function=sigmoid) -> None:
    self.input_nodes =  []
    self.output_nodes =  []
    self.weights = []
    self.activation_function = activation_function
    self.deriv_activation_function = deriv_activation_functions.get(activation_function)
    self.delta = None

  def dot_product(self):
    """
    Calculates the dot product of the inputs and weights.

    """
    lista_input =  [nodo.get_output() for nodo in self.input_nodes]
    return np.dot(lista_input,self.weights)

  def get_output(self):
    """
    Calculates the output of the neuron using the activation function.

    """
    return self.activation_function(self.dot_product())

  def connect_forward(self, output_node):
    """
    Connects the current node to a forward node.

    Parameter:
        output_node: The node to connect to.

    Returns:
        Node: Self node.
    """
    self.output_nodes.append(output_node)
    output_node.add_input_node(self)
    return self

  def add_input_node(self, node):
    """
    Adds an input node to the neuron.

    Parameter:
        Node: The input node to add.

    """
    self.input_nodes.append(node)
    # Recalculate weights with mean=0 and std=1 when adding a node
    self.weights= np.random.randn(len(self.input_nodes))
    return self

  def get_delta(self):
    """
    Returns the delta value of the neuron, which is used in backpropagation.

    """
    return self.delta

# ------VISUALIZATION--------
  def print(self):
    """
    Displays info of Neuron
    """
    lista_input =  [nodo.get_output() for nodo in self.input_nodes]
    for index in range(len(lista_input)):
        print(f"{lista_input[index]} x {self.weights[index]} = {lista_input[index]*self.weights[index]}")
    print(f"Dot product: {self.dot_product()}")
    print(f"Output: {self.get_output()}")

  def print_structure(self,iteration):
    """
    Displays neural network structure
    """
    tab = iteration*"\t"
    print(f"{tab}Iteration:{iteration}, Weights: {self.weights}")
    iteration+=1
    for input_node in self.input_nodes:
      input_node.print_structure(iteration)

# ------BACKPROPAGATION--------
  def backpropagate(self, learning_rate=0.1):
    """
    Performs backpropagation process recursively down to the OnlyValueNode nodes.
    """
    self.update_weights(learning_rate)
    for input_node in self.input_nodes:
      input_node.hidden_delta_function()
      input_node.backpropagate()


  def update_weights(self, learning_rate=0.1):
    """
    Recalculates the weights using the following formula for the layer: weight = weight - (learning_rate * output_backward * delta)
    """
    for i in range(len(self.input_nodes)):
      self.weights[i] = self.weights[i] - (learning_rate* self.input_nodes[i].get_output()*self.delta)



  def output_delta_function(self,value):
    """
    Calculates delta error for the output neuron: (prediction-value) * deriv_activation_function(prediction).
    """
    deriv_cost_function = self.get_output() - value
    self.delta = deriv_cost_function * self.deriv_activation_function(self.get_output())
    return self.delta

  def hidden_delta_function(self):
    """
    Calculates the delta in a neuron, as the errors it propagates, using the formula: Summation( weight_forward * delta_forward) * deriv_activation_function(prediction).
    """
    suma = 0
    for node in self.output_nodes:
      weight_forward = node.get_weight_of_node(self)
      delta_forward =  node.get_delta()
      suma += (weight_forward * delta_forward)

    self.delta = suma * self.deriv_activation_function(self.get_output())
    return self.delta


  def get_weight_of_node(self, node):
    """
    Returns the weight between 2 nodes
    """
    for i in range(len(self.input_nodes)):
      if (self.input_nodes[i]==node):
        return self.weights[i]
    return None

In [37]:
class OnlyValueNode(Node):
  """
  Represents a node that holds a single input value in a neural network.
  """
  def __init__(self, input_value) -> None:
    self.input_value = input_value
    self.output_nodes =  []

  def get_output(self):
     """
     Returns the input value
     """
     return self.input_value

  def connect_forward(self, output_node):
    self.output_nodes.append(output_node)
    output_node.add_input_node(self)
    return self

  def add_input_node(self,node):
    return Exception(f"Input node: {self}, does not have inputs")

  def hidden_delta_function(self):
    pass

  def backpropagate(self):
    pass

  def print_structure(self,iteration):
    tab = iteration*"\t"
    print(f"{tab}Input Value:{self.input_value}")

## FeedForward and Backpropagation Example

In [40]:
# Represents a single example in a dataset: Independent vars:[0,1] Dependent: [1]
input_nodes = [OnlyValueNode(0),OnlyValueNode(1)]
output_dataset = [1]

# -----------FeedForward---------------
# Create a layer and connect it with inputs
layer_1 =  [Neuron(),Neuron()]
connect_layer_forward(input_nodes,layer_1)

# Create 2nd layer and connect with the previous one
layer_2 = [Neuron()]
connect_layer_forward(layer_1,layer_2)

# Result of the neuron in the last layer 

print(f"Prediction: {layer_2[0].get_output()}, Real:{output_dataset[0]}")
last_neuron = layer_2[0]

# ----------Backpropagating------------

for i in range(101):
  delta = last_neuron.output_delta_function(output_dataset[0])
  last_neuron.backpropagate(1)
  if i%10==0:
    print(f"it:{i} Prediction: {last_neuron.get_output()}, Real:{output_dataset[0]}, Delta: {last_neuron.delta}, Output: {last_neuron.get_output()}")


Prediction: 0.3086942982845839, Real:1
it:0 Prediction: 0.32285336622688776, Real:1, Delta: -0.1058617467171313, Output: 0.32285336622688776
it:10 Prediction: 0.4932419228665639, Real:1, Delta: -0.12239002182171373, Output: 0.4932419228665639
it:20 Prediction: 0.666480261734126, Real:1, Delta: -0.10971076599114873, Output: 0.666480261734126
it:30 Prediction: 0.7844282463754476, Real:1, Delta: -0.08299900120741942, Output: 0.7844282463754476
it:40 Prediction: 0.8525511619976301, Real:1, Delta: -0.06107647744537485, Output: 0.8525511619976301
it:50 Prediction: 0.8923782477681096, Real:1, Delta: -0.046243981744790406, Output: 0.8923782477681096
it:60 Prediction: 0.9171711013548937, Real:1, Delta: -0.03632709265160451, Output: 0.9171711013548937
it:70 Prediction: 0.9336184344377849, Real:1, Delta: -0.029481197443423274, Output: 0.9336184344377849
it:80 Prediction: 0.9451305112441519, Real:1, Delta: -0.024570681477210237, Output: 0.9451305112441519
it:90 Prediction: 0.9535460084029863, Real